In [1]:
import pandas as pd

from run import Dataset

DATASET = Dataset.SPX_US

config = DATASET.value()
config

(CVXPY) Jun 12 07:38:11 PM: Encountered unexpected exception importing solver PROXQP:
ImportError("dlopen(/Users/buchkovv/qamsi/.venv/lib/python3.12/site-packages/cmeel.prefix/lib/python3.12/site-packages/proxsuite/proxsuite_pywrap.cpython-312-darwin.so, 0x0002): Library not loaded: @rpath/libc++.1.dylib\n  Referenced from: <73C5C23C-530A-3E0E-A88E-7897A0C69618> /Users/buchkovv/qamsi/.venv/lib/python3.12/site-packages/cmeel.prefix/lib/python3.12/site-packages/proxsuite/proxsuite_pywrap.cpython-312-darwin.so\n  Reason: tried: '/Users/runner/miniconda3/envs/proxsuite/lib/libc++.1.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/runner/miniconda3/envs/proxsuite/lib/libc++.1.dylib' (no such file), '/var/folders/0j/bwqcs4y508s2n4ck4dhf3rpc0000gn/T/cmeel-qko5evau/whl/cmeel.prefix/lib/libc++.1.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/var/folders/0j/bwqcs4y508s2n4ck4dhf3rpc0000gn/T/cmeel-qko5evau/whl/cmeel.prefix/lib/libc++.1.dylib' (no such file), '/Users

ExperimentConfig:
* RANDOM_SEED = 12
* PATH_OUTPUT = /Users/buchkovv/qamsi/data/output
* SAVE_PATH = /Users/buchkovv/qamsi/backtests/runs
* DF_FILENAME = spx_data.csv
* START_DATE = 2000-02-08 00:00:00
* END_DATE = 2024-12-31 00:00:00
* REBALANCE_FREQ = 7
* HEDGE_FREQ = 1
* N_LOOKBEHIND_PERIODS = 252
* MIN_ROLLING_PERIODS = 252
* ASSET_UNIVERSE = ('spx',)
* FACTORS = ('spx',)
* TARGETS = ('vol', 'naive_vol', 'l_shrinkage')
* HEDGING_ASSETS = ('spx_fut',)
* RF_NAME = acc_rate
* MKT_NAME = spx
* FRACTIONAL_DIFFERENCING = 0.5
* PATH_FEATURES = /data/gw
* PATH_MARKET_DATA = /data/spx_stocks
* PATH_BETTER_MARKET_DATA = /data/spxc
* PATH_INPUT = /Users/buchkovv/qamsi/data/input
* INITIAL_DF_FILENAME = initial_df.csv
* JKP_DATA_FILENAME = jkp_data.csv
* STOCKS_LIST_FILENAME = spx_stocks_list.csv
* INITIAL_FEATURES_FILENAME = initial_features_df.csv
* RETURNS_FILENAME = returns_incl_div_consituents_w_name.csv
* BETTER_RETURNS_FILENAME = returns_data_cleaned_better.parquet
* PRESENCE_MATRIX_FIL

In [2]:
from pathlib import Path

PATH = Path("../../data/output")

data = pd.read_csv(PATH / "spx_data.csv")
data["date"] = pd.to_datetime(data["date"])
data = data.set_index("date")
data.shape

(4980, 2087)

In [3]:
stocks = list(
    tuple(
        pd.read_csv(config.PATH_OUTPUT / config.STOCKS_LIST_FILENAME)
        .iloc[:, 0]
        .astype(str)
        .tolist(),
    )
)

ret = data[stocks]
ret.shape

(4980, 1033)

### 1. Avg Corr.

In [4]:
# import pandas as pd
#
# # Assuming `ret` is your DataFrame of returns (date as index), and it is already loaded
#
# # Define the rolling window and minimum periods
# window = 252
# min_periods = 252
#
# # Function to compute the average of non-diagonal elements in each correlation matrix
# def avg_non_diagonal_elements(corr_matrix):
#     # Select the non-diagonal elements using numpy
#     non_diag = corr_matrix.values[np.triu_indices_from(corr_matrix, k=1)]
#     return np.nanmean(non_diag)
#
# # Function to compute rolling correlation
# def rolling_avg_non_diag_corr(df, window, min_periods):
#     # Initialize a list to store results
#     results = []
#
#     # Perform calculation for each rolling window
#     for start in range(len(df) - window + 1):
#         end = start + window
#         rolling_window = df.iloc[start:end]
#
#         # Compute the correlation matrix for the rolling window
#         corr_matrix = rolling_window.corr()
#
#         # Compute the average of non-diagonal elements
#         avg_correlation = avg_non_diagonal_elements(corr_matrix)
#         results.append(avg_correlation)
#
#     # Create a series with the results
#     rolling_avg_corr = pd.Series(results, index=df.index[window - 1 :])
#     return rolling_avg_corr
#
# # Calculate rolling average correlation of non-diagonal elements
# rolling_avg_corr = rolling_avg_non_diag_corr(ret, window, min_periods)
#
# # Output rolling_avg_corr as the result
# rolling_avg_corr.shape

In [5]:
rolling_avg_corr = pd.read_csv(PATH / "avg_corr.csv")

In [6]:
# rolling_avg_corr.to_csv(PATH / "avg_corr.csv")

### 2. Average volatility.

In [7]:
avg_vol = ret.rolling(window=252, min_periods=1).std().mean(axis=1)
avg_vol.shape

(4980,)

### 3. EW Portfolio.

In [8]:
import numpy as np

ew = ret.apply(np.nanmean, axis=1)
ew

date
2004-03-19   -0.008731
2004-03-22   -0.014426
2004-03-23   -0.000700
2004-03-24   -0.002553
2004-03-25    0.018077
                ...   
2023-12-22    0.003390
2023-12-26    0.005998
2023-12-27    0.001137
2023-12-28    0.001180
2023-12-29   -0.002990
Length: 4980, dtype: float64

In [9]:
ewma = []
for date in ew.index[21 - 1 :]:
    end = date
    sample = ew.loc[:end].iloc[-21:]

    ma = sample.ewm(alpha=0.1).mean().iloc[-1]

    ewma.append([date, ma])

In [10]:
ewma = pd.DataFrame(ewma, columns=["date", "ewma"]).set_index("date").iloc[:, 0]
ewma

date
2004-04-19    0.000462
2004-04-20   -0.001112
2004-04-21   -0.000060
2004-04-22    0.001986
2004-04-23    0.001667
                ...   
2023-12-22    0.003470
2023-12-26    0.003755
2023-12-27    0.003549
2023-12-28    0.003336
2023-12-29    0.002619
Name: ewma, Length: 4960, dtype: float64

### 4. Ledoit-Wolf Shrinkage Intensity.

In [11]:
from tqdm import tqdm
from sklearn.covariance import LedoitWolf

lw_intensity = []
for date in tqdm(ew.index[252 - 1 :]):
    end = date
    sample = ret.loc[:end].iloc[-252:].dropna(axis=1)

    lw = LedoitWolf()
    lw.fit(sample)

    lw_intensity.append([date, lw.shrinkage_])
lw_intensity = (
    pd.DataFrame(lw_intensity, columns=["date", "lw_intensity"])
    .set_index("date")
    .iloc[:, 0]
)

100%|██████████| 4729/4729 [01:47<00:00, 43.81it/s]


In [12]:
lw_intensity.head()

date
2005-03-18    0.067181
2005-03-21    0.067755
2005-03-22    0.068819
2005-03-23    0.068787
2005-03-24    0.068928
Name: lw_intensity, dtype: float64

In [13]:
sample.shape

(252, 432)

In [14]:
np.nanmean(np.where(sample, sample > 0, 1), axis=0).mean()

np.float64(0.5251322751322751)

### 5. Momentum

In [15]:
momentum = []
for date in tqdm(ew.index[21 - 1 :]):
    end = date
    sample = ret.loc[:end].iloc[-21:].dropna(axis=1)

    mom = np.nanmean(np.where(sample, sample > 0, 1), axis=0).mean()

    momentum.append([date, mom])
momentum = (
    pd.DataFrame(momentum, columns=["date", "momentum_feature"])
    .set_index("date")
    .iloc[:, 0]
)

100%|██████████| 4960/4960 [00:01<00:00, 4049.35it/s]


### 6. Trace.

In [16]:
np.trace(sample.cov())

np.float64(0.12288092370262858)

In [17]:
trace = []
for date in tqdm(ew.index[252 - 1 :]):
    end = date
    sample = ret.loc[:end].iloc[-252:].dropna(axis=1)

    mom = np.trace(sample.cov())

    trace.append([date, mom])
trace = pd.DataFrame(trace, columns=["date", "trace"]).set_index("date").iloc[:, 0]

100%|██████████| 4729/4729 [00:04<00:00, 1141.05it/s]


### 7. Universe Volatility.

In [18]:
ew_vol = ew.rolling(window=252, min_periods=1).std()
ew_vol

date
2004-03-19         NaN
2004-03-22    0.004027
2004-03-23    0.006896
2004-03-24    0.006244
2004-03-25    0.012291
                ...   
2023-12-22    0.009123
2023-12-26    0.009099
2023-12-27    0.009090
2023-12-28    0.009090
2023-12-29    0.009049
Length: 4980, dtype: float64

### Append features.

In [19]:
rolling_avg_corr["date"] = pd.to_datetime(rolling_avg_corr["date"])
rolling_avg_corr = rolling_avg_corr.set_index("date")
rolling_avg_corr.columns = ["avg_corr_stocks"]

In [20]:
features = rolling_avg_corr.merge(
    avg_vol.rename("stocks_vol"), how="inner", left_index=True, right_index=True
)
features.shape

(4729, 2)

In [21]:
features = features.merge(ewma, how="inner", left_index=True, right_index=True)
features.shape

(4729, 3)

In [22]:
features = features.merge(lw_intensity, how="inner", left_index=True, right_index=True)
features.shape

(4729, 4)

In [23]:
features = features.merge(momentum, how="inner", left_index=True, right_index=True)
features.shape

(4729, 5)

In [24]:
features = features.merge(trace, how="inner", left_index=True, right_index=True)
features.shape

(4729, 6)

In [25]:
features = features.merge(
    ew_vol.rename("universe_vol"), how="inner", left_index=True, right_index=True
)
features.shape

(4729, 7)

In [26]:
import pandas as pd

targets = pd.read_csv(PATH / "linear_targets.csv")
targets["date"] = pd.to_datetime(targets["date"])
targets = targets.set_index("date")

In [27]:
# dnk_features = targets.loc[:, ["l_shrinkage", "vol"]]
# dnk_features = dnk_features.rename(columns={"l_shrinkage": "l_shrinkage_feat"})
# rolling_optimal = dnk_features["l_shrinkage_feat"].rolling(window=252, min_periods=1).mean().rename("l_shrinkage_mu")
# rolling_vol = dnk_features["l_shrinkage_feat"].rolling(window=252, min_periods=1).std().rename("l_shrinkage_vol").fillna(0)
# rolling_vol

date
2005-03-18    0.000000
2005-03-21    0.028284
2005-03-22    0.023094
2005-03-23    0.090323
2005-03-24    0.100896
                ...   
2023-11-17    0.289817
2023-11-20    0.290356
2023-11-21    0.290139
2023-11-22    0.289934
2023-11-24    0.289859
Name: l_shrinkage_vol, Length: 4704, dtype: float64

In [28]:
# add_feat = dnk_features[["l_shrinkage_feat"]].merge(rolling_optimal, how="left", left_index=True, right_index=True)
# add_feat = add_feat.merge(rolling_vol, how="left", left_index=True, right_index=True)
# add_feat.shape

(4716, 3)

In [29]:
# new_data = pd.merge_asof(data, add_feat, left_index=True, right_index=True, tolerance=pd.Timedelta("1D"))
# new_data.shape

(4980, 2090)

In [30]:
new_data = pd.merge_asof(
    data, targets, left_index=True, right_index=True, tolerance=pd.Timedelta("1D")
)
new_data = pd.merge_asof(
    new_data, features, left_index=True, right_index=True, tolerance=pd.Timedelta("1D")
)
new_data.shape

(4980, 2100)

In [32]:
new_data.to_csv(PATH / "spx_data.csv")